In [ ]:
# Import all libraries

import numpy as np
from sklearn.cluster import DBSCAN
from UTILS import find_consecutive_instances, hydrogenBondCheckGivenIndices, calculateAtomTypes, BARMCNaive
from CMC import CMC
from hyperparameters import Hyperparameters
import time

In [ ]:
# # All atoms as input
# # Brute Force

fileLocation = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\Parsed Data\convoy paper\data\allAtomsWithRings.csv" #location for information of atoms

filePath = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\Parsed Data\convoy paper\data\dcd2-55k-65k-fixed.npy"

data = np.load(filePath)

In [ ]:
#define the hyperparameters
dataSize = [500]
Ks = np.arange(0, 10, 1)
Ms = [25]
starterFrames = [379]
t_1s = [2]
t_2s = [0.25]
epsilons = [3.5]


for varyk in Ks:
    for varyM in Ms:
        for size in dataSize:
            for startFrame in starterFrames:
                for t_1 in t_1s:
                    for t_2 in t_2s:
                        for eps in epsilons:
                            hyperParameters = Hyperparameters(m = varyM, k = varyk, epsilon = eps, t1 = t_1,  t2 = t_2, threshold=4)
                            data = np.load(filePath)
                            print(f"K {varyk + 1}, M {varyM}, S {size}, t1 = {hyperParameters.t1}, t2 = {hyperParameters.t2}, epsilon = {hyperParameters.epsilon}, threshold = {hyperParameters.threshold}, DATA = {[startFrame, startFrame+size]}")


                            orData = np.transpose(data[0:size], (1,0,2)).tolist()


                            totalTime = 0

                            t1 = time.time()

                            clustering_clf = DBSCAN(eps=hyperParameters.epsilon)



                            clf = CMC(clustering_clf, k=hyperParameters.k, m=hyperParameters.m)

                            convoys = clf.fit_predict(orData)

                            data = data[startFrame:startFrame+size]


                            sorted_list = sorted(convoys, key=lambda x: x.start_time)

                            atomTypes = calculateAtomTypes(fileLocation)
                            finalConvoys = {}
                            for index, convoy in enumerate(sorted_list):

                                finalConvoys[index] = None

                                arr = hydrogenBondCheckGivenIndices(convoy.start_time, convoy.end_time, convoy.indices, data, atomTypes)

                                finalConvoys[index] = [arr, convoy]

                            for key, (indices, convoy) in finalConvoys.items():

                                df_ranges = indices

                                consecutive_instances = find_consecutive_instances(df_ranges, hyperParameters.k)

                                final = BARMCNaive(consecutive_instances, hyperParameters.t1, hyperParameters.t2, k=hyperParameters.k)

                                if not final.empty:
                                    total = final['end'] - final['start'] + 1
                                    print(f"Total: {total.sum()}")
                                    display(final.sort_values(by='start'))


                            totalTime += time.time() - t1
                            print(f"Total time {totalTime}")

